# Convolutional Neural Network with Imaging Data

This notebook provides a CNN model with VGG architecutre to train MRI brain data to classify CDR diagnosis results.



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#read data
import numpy as np
train_x = np.load("/content/gdrive/MyDrive/biostat625Final/image_train_x_cut_middle.npy")
train_y = np.load("/content/gdrive/MyDrive/biostat625Final/image_train_y_cut_middle.npy")
test_x = np.load("/content/gdrive/MyDrive/biostat625Final/image_test_x_cut_middle.npy")
test_y = np.load("/content/gdrive/MyDrive/biostat625Final/image_test_y_cut_middle.npy")
  

In [4]:
#reshape 
train_x = train_x.reshape((160,176,208,88,1))
test_x = test_x.reshape((38,176,208,88,1))

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
#divide by train and validation sets
from tensorflow.data import Dataset
count_training=128
train_ds = Dataset.from_tensor_slices((train_x[:count_training], train_y[:count_training]))
validation_ds = Dataset.from_tensor_slices((train_x[count_training:], train_y[count_training:]))
test_ds = Dataset.from_tensor_slices((test_x, test_y)) 

In [62]:
#define model
def get_model():
    inputs = keras.Input((176,208,88,1))
    x = layers.Conv3D(filters=8, kernel_size=3, activation="relu")(inputs)
    x = layers.Conv3D(filters=8, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.Conv3D(filters=16, kernel_size=3, activation='relu')(x)
    x = layers.Conv3D(filters=16, kernel_size=3, activation='relu')(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.Conv3D(filters=32, kernel_size=3, activation='relu')(x)
    x = layers.Conv3D(filters=32, kernel_size=3, activation='relu')(x)
    x = layers.Conv3D(filters=32, kernel_size=3, activation='relu')(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(units=128, activation="relu")(x)
    x = layers.Dense(units=64, activation="relu")(x)
    x = layers.Dense(units=4, activation="relu")(x)
    outputs = layers.Dense(units=1, activation="sigmoid")(x)
    
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model
  
model = get_model()
model.summary()



Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 176, 208, 88, 1)  0         
                             ]                                   
                                                                 
 conv3d_149 (Conv3D)         (None, 174, 206, 86, 8)   224       
                                                                 
 max_pooling3d_72 (MaxPoolin  (None, 87, 103, 43, 8)   0         
 g3D)                                                            
                                                                 
 conv3d_150 (Conv3D)         (None, 85, 101, 41, 16)   3472      
                                                                 
 conv3d_151 (Conv3D)         (None, 83, 99, 39, 16)    6928      
                                                                 
 max_pooling3d_73 (MaxPoolin  (None, 41, 49, 19, 16)   0     

In [63]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.metrics import SpecificityAtSensitivity
from numpy.random import seed
METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      SpecificityAtSensitivity(sensitivity=0.8, name='sensitivity'),
]
seed(1)
#run with GPU
with tf.device('/device:GPU:0'):
# Compile model.
  initial_learning_rate = 0.0001
  lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
  )
  model.compile(
      loss="binary_crossentropy",
      optimizer=keras.optimizers.Adam(learning_rate=0.0001),
      metrics=METRICS,
  )

  # Define callbacks.
  checkpoint_cb = keras.callbacks.ModelCheckpoint(
      "3d_image_classification.h5", save_best_only=True
  )
  
  # Fit model
  epochs = 100
  model.fit(
      train_ds.batch(batch_size=batch_size),
      validation_data=validation_ds.batch(batch_size=batch_size),
      epochs=epochs,
      shuffle=True,
      verbose=2,
      callbacks=[checkpoint_cb],
  )

Epoch 1/100
8/8 - 12s - loss: 0.7101 - tp: 22.0000 - fp: 31.0000 - tn: 35.0000 - fn: 40.0000 - accuracy: 0.4453 - precision: 0.4151 - recall: 0.3548 - auc: 0.4453 - sensitivity: 0.1364 - val_loss: 0.6930 - val_tp: 17.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 1.0000 - val_accuracy: 0.5312 - val_precision: 0.5484 - val_recall: 0.9444 - val_auc: 0.4802 - val_sensitivity: 0.0000e+00 - 12s/epoch - 2s/step
Epoch 2/100
8/8 - 9s - loss: 0.6867 - tp: 35.0000 - fp: 26.0000 - tn: 40.0000 - fn: 27.0000 - accuracy: 0.5859 - precision: 0.5738 - recall: 0.5645 - auc: 0.5968 - sensitivity: 0.2424 - val_loss: 0.6939 - val_tp: 14.0000 - val_fp: 13.0000 - val_tn: 1.0000 - val_fn: 4.0000 - val_accuracy: 0.4688 - val_precision: 0.5185 - val_recall: 0.7778 - val_auc: 0.3829 - val_sensitivity: 0.0000e+00 - 9s/epoch - 1s/step
Epoch 3/100
8/8 - 9s - loss: 0.6484 - tp: 44.0000 - fp: 26.0000 - tn: 40.0000 - fn: 18.0000 - accuracy: 0.6562 - precision: 0.6286 - recall: 0.7097 - auc: 0.6860 - sensitivit

In [64]:
score_test = model.evaluate(test_ds.batch(batch_size))
for name, value in zip(model.metrics_names, score_test):
    print(name, ': ', value)

3/3 [==============================] - 2s 278ms/step - loss: 1.3984 - tp: 8.0000 - fp: 3.0000 - tn: 15.0000 - fn: 12.0000 - accuracy: 0.6053 - precision: 0.7273 - recall: 0.4000 - auc: 0.6278 - sensitivity: 0.5000
loss :  1.3983997106552124
tp :  8.0
fp :  3.0
tn :  15.0
fn :  12.0
accuracy :  0.6052631735801697
precision :  0.7272727489471436
recall :  0.4000000059604645
auc :  0.6277777552604675
sensitivity :  0.5
